# Embedding delle Frasi

In [57]:
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

import numpy as np
import re
import string
import os
import operator
import pickle

import random
import pandas as pd

In [2]:
def build_dictionary(sentences):
    d = dict()
    index = 0
    for sentence in sentences:
        for word in sentence:
            if not word in d:
                d[word] = index
                index += 1
    return d

def word_counts(sentences):
    d = dict()
    for sentence in sentences:
        for word in sentence:
            if not word in d:
                d[word] = 1
            else:
                d[word] += 1
    return d

def rev_list(d):
    rd = [None]*len(d)
    for w,i in d.items():
        rd[i] = w
    return rd

def rev_dict(d):
    rd = dict()
    for w,i in d.items():
        rd[i] = w
    return rd

def words_to_ints(words,d):
    return [d[w] for w in d]

def sentences_to_int_lists(sentences,d):
    return [words_to_ints(words) for words in sentences]


###### Creazione del dataset come sottoinsieme bilanciato dei documenti

In [3]:
csv_filename = '../atti.csv'
model_filename = 'gensim_model.d2v'

In [4]:
df = pd.read_csv(csv_filename)

In [5]:
size_nc = len(df.loc[df['label'] == 'non_costitutivo'].groupby('filename'))
size_nc

7246

In [7]:
grouped = df.loc[df['label'] == 'costitutivo'].groupby(df["filename"])
dfs = [g[1] for g in list(grouped)[:size_nc]]

In [8]:
grouped_nc = df.loc[df['label'] == 'non_costitutivo'].groupby(df["filename"])
dfs_nc = [g[1] for g in list(grouped_nc)]

In [9]:
df_balanced = pd.concat(dfs + dfs_nc)

In [1]:
df_balanced

In [11]:
pd_sentences = df_balanced['sentence']

###### Creazione degli embedding

In [12]:
class LabeledLineSentence(object):
    '''Pass the pandas column'''
    def __init__(self, sentences):
        self.sentences = sentences
    def __iter__(self):
        i = 0
        for line in sentences:
            yield LabeledSentence(utils.to_unicode(line).split(), ['SENT_%s' % i])
            i += 1

class LabeledLineSentence(object):
    def __init__(self, df):
        self.df = df
        
        flipped = {}
        
        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')
    
    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])
    
    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
        return self.sentences
    
    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences
            
def iter_sentences(sents):
    i = 0
    for line in sents:
        yield LabeledSentence(line, ['SENT_%s' % i])
        i += 1

def randomly(seq):
    shuffled = list(seq)
    random.shuffle(shuffled)
    return iter(shuffled)

#sentences = iter_sentences(pd_sentences) #LabeledLineSentence(pd_sentences)

In [13]:
# Modello dell'embedding

def build_embedding(sentences):
    if os.path.exists(model_filename):
        model = Doc2Vec.load(model_filename)
    else:
        model = Doc2Vec(min_count=1, window=10, size=100, sample=1e-5, negative=5, workers=2)
        model.build_vocab(sentences)
        print 'Vocabulary built'
        #for epoch in range(5):
        #    print 'Epoch', epoch
        model.train(sentences, model.corpus_count, epochs = 5)
        model.save(model_filename)
        print 'Model saved'
    return model

In [14]:
d = build_dictionary(s.split() for s in pd_sentences)

In [15]:
len(d)

138822

In [16]:
def first_n_words(dictionary, n):
    rd = rev_dict(d)
    wc = word_counts(s.split() for s in pd_sentences)
    sorted_wc = sorted(wc.items(), key=operator.itemgetter(1))
    return set(reversed([x[0] for x in sorted_wc[-n:]]))

In [17]:
first_10000_words = first_n_words(d, 10000)

In [18]:
def substitute_word(word, permitted_words, unknown = 'UNK'):
    return word if word in permitted_words else unknown

def reduced_sentence(sentence, permitted_words):
    return [substitute_word(word, permitted_words) for word in sentence]

def reduce_dictionary(sentences, permitted_words, min_words=2):
    for sentence in sentences:
        new_sentence = reduced_sentence(sentence, permitted_words)
        if len(new_sentence) >= min_words:
            yield new_sentence


In [21]:
filtered_sentences = reduce_dictionary((s.split() for s in pd_sentences), first_10000_words)
filtered_sentences_list = list(filtered_sentences)

In [23]:
# Esecuzione del modello

model = build_embedding(list(iter_sentences(filtered_sentences_list)))

Vocabulary built
Model saved


In [24]:
model.most_similar('atto')

[('presente', 0.8062084913253784),
 ('apportare', 0.7802978754043579),
 ('soppressioni', 0.7477972507476807),
 ('allega', 0.7333101034164429),
 ('aggiunte', 0.7322391271591187),
 ('statuto', 0.731306791305542),
 ('integrante', 0.7226213216781616),
 ('regolata', 0.7120342254638672),
 ('forza', 0.7065362930297852),
 ('allegano', 0.7060737609863281)]

In [25]:
def sentence_vector(model, sentence, permitted_words):
    return model.infer_vector(reduced_sentence(sentence.split(' '), permitted_words))

In [48]:
# Esempio di sentence vector

sv = sentence_vector(model, pd_sentences[551068], first_10000_words)
sv

array([-0.00877847,  0.08421552, -0.00934239, -0.06026119, -0.0486962 ,
       -0.10909535,  0.01427933,  0.0382507 ,  0.06857342, -0.05685881,
       -0.01122126,  0.05149804, -0.07070133,  0.01299185,  0.08797149,
       -0.02599948,  0.00463746, -0.02856188,  0.00580665,  0.01292427,
        0.0679419 ,  0.0067199 ,  0.00769502,  0.07473454,  0.02272988,
       -0.02177911, -0.05640026,  0.08370669,  0.01138843, -0.07007347,
       -0.08425567, -0.00059223, -0.0390888 ,  0.0140768 , -0.11788081,
        0.01243659, -0.06523187,  0.02117974, -0.00639699, -0.00492997,
        0.01472021, -0.0201715 ,  0.00113072,  0.01332173, -0.02123491,
       -0.01164006, -0.00858516,  0.06819962, -0.02509951,  0.02500732,
        0.0713005 , -0.09450735, -0.04908381,  0.03631534, -0.08748867,
        0.06206969,  0.00515663,  0.02944721,  0.0006274 ,  0.0327379 ,
        0.0844673 , -0.06255493, -0.03983356, -0.07354014, -0.02518529,
        0.05504263,  0.06384774, -0.05127484, -0.00868062,  0.00

###### Costruzione del dataset

In [49]:
def build_dataset(model, df, permitted_words):
    #embeddings = np.array([sentence_vector(model, sentence, permitted_words) for sentence in sentences])
    filename = ""
    docs = []
    labels = []
    curdoc = []                  # lista delle frasi del documento corrente
    for i in xrange(len(df)):
        row = df.iloc[i] 
        if filename == "":
            filename = row["filename"]
            labels.append(row["label"])
            
        embedding = sentence_vector(model, row['sentence'], permitted_words)
        if filename == row["filename"]:
            curdoc.append(embedding)
        else:
            print "%s with len: %d" % (filename, len(curdoc))
            docs.append(curdoc)
            curdoc = [embedding]
            labels.append(row["label"])
            filename = row['filename']
    if len(curdoc)>0:
        docs.append(curdoc)
    return docs, labels

In [2]:
docs, labels = build_dataset(model, df_balanced, first_10000_words)

In [53]:
label_map = {'costitutivo':1, 'non_costitutivo':0}
labels_n = [label_map[l] for l in labels]

In [58]:
with open("embedded_docs.p", "w") as fout:
    pickle.dump([docs, labels_n], fout)